In [1]:
import torch
import torch.onnx
import torch.nn as nn
import torch.nn.init as init
import torch.utils.model_zoo as model_zoo
from pcdet.config import cfg, cfg_from_yaml_file
from pcdet.models import build_network
from pcdet.datasets import build_dataloader
from pcdet.utils import common_utils
import os
import numpy as np
import torch
# # import onnx
import onnxruntime as ort
import torch.nn as nn

# from typing import Sequence, NamedTuple



In [2]:
class AllDSVTBlocksTRT(nn.Module):
    def __init__(self, dsvtblocks_list, layer_norms_list):
        super().__init__()
        self.layer_norms_list = layer_norms_list
        self.dsvtblocks_list = dsvtblocks_list
    def forward(
        self,
        pillar_features,
        set_voxel_inds_tensor_shift_0,
        set_voxel_inds_tensor_shift_1,
        set_voxel_masks_tensor_shift_0,
        set_voxel_masks_tensor_shift_1,
        pos_embed_tensor,
    ):
        outputs = pillar_features

        residual = outputs
        blc_id = 0
        set_id = 0
        set_voxel_inds = set_voxel_inds_tensor_shift_0[set_id:set_id+1].squeeze(0)
        set_voxel_masks = set_voxel_masks_tensor_shift_0[set_id:set_id+1].squeeze(0)
        pos_embed = pos_embed_tensor[blc_id:blc_id+1, set_id:set_id+1].squeeze(0).squeeze(0)
        # inputs = (outputs, set_voxel_inds, set_voxel_masks, pos_embed, True)
        inputs = (outputs, set_voxel_inds, set_voxel_masks, pos_embed)

        outputs = self.dsvtblocks_list[blc_id].encoder_list[set_id](*inputs)
        set_id = 1
        set_voxel_inds = set_voxel_inds_tensor_shift_0[set_id:set_id+1].squeeze(0)
        set_voxel_masks = set_voxel_masks_tensor_shift_0[set_id:set_id+1].squeeze(0)
        pos_embed = pos_embed_tensor[blc_id:blc_id+1, set_id:set_id+1].squeeze(0).squeeze(0)
        # inputs = (outputs, set_voxel_inds, set_voxel_masks, pos_embed, True)
        inputs = (outputs, set_voxel_inds, set_voxel_masks, pos_embed)

        outputs = self.dsvtblocks_list[blc_id].encoder_list[set_id](*inputs)

        outputs = self.layer_norms_list[blc_id](residual + outputs)

        residual = outputs
        blc_id = 1
        set_id = 0
        set_voxel_inds = set_voxel_inds_tensor_shift_1[set_id:set_id+1].squeeze(0)
        set_voxel_masks = set_voxel_masks_tensor_shift_1[set_id:set_id+1].squeeze(0)
        pos_embed = pos_embed_tensor[blc_id:blc_id+1, set_id:set_id+1].squeeze(0).squeeze(0)
        inputs = (outputs, set_voxel_inds, set_voxel_masks, pos_embed)
        # inputs = (outputs, set_voxel_inds, set_voxel_masks, pos_embed, True)

        outputs = self.dsvtblocks_list[blc_id].encoder_list[set_id](*inputs)
        set_id = 1
        set_voxel_inds = set_voxel_inds_tensor_shift_1[set_id:set_id+1].squeeze(0)
        set_voxel_masks = set_voxel_masks_tensor_shift_1[set_id:set_id+1].squeeze(0)
        pos_embed = pos_embed_tensor[blc_id:blc_id+1, set_id:set_id+1].squeeze(0).squeeze(0)
        inputs = (outputs, set_voxel_inds, set_voxel_masks, pos_embed)
        # inputs = (outputs, set_voxel_inds, set_voxel_masks, pos_embed, True)
        outputs = self.dsvtblocks_list[blc_id].encoder_list[set_id](*inputs)

        outputs = self.layer_norms_list[blc_id](residual + outputs)

        residual = outputs
        blc_id = 2
        set_id = 0
        set_voxel_inds = set_voxel_inds_tensor_shift_0[set_id:set_id+1].squeeze(0)
        set_voxel_masks = set_voxel_masks_tensor_shift_0[set_id:set_id+1].squeeze(0)
        pos_embed = pos_embed_tensor[blc_id:blc_id+1, set_id:set_id+1].squeeze(0).squeeze(0)
        # inputs = (outputs, set_voxel_inds, set_voxel_masks, pos_embed, True)
        inputs = (outputs, set_voxel_inds, set_voxel_masks, pos_embed)
        outputs = self.dsvtblocks_list[blc_id].encoder_list[set_id](*inputs)
        set_id = 1
        set_voxel_inds = set_voxel_inds_tensor_shift_0[set_id:set_id+1].squeeze(0)
        set_voxel_masks = set_voxel_masks_tensor_shift_0[set_id:set_id+1].squeeze(0)
        pos_embed = pos_embed_tensor[blc_id:blc_id+1, set_id:set_id+1].squeeze(0).squeeze(0)
        # inputs = (outputs, set_voxel_inds, set_voxel_masks, pos_embed, True)
        inputs = (outputs, set_voxel_inds, set_voxel_masks, pos_embed)
        outputs = self.dsvtblocks_list[blc_id].encoder_list[set_id](*inputs)

        outputs = self.layer_norms_list[blc_id](residual + outputs)

        residual = outputs
        blc_id = 3
        set_id = 0
        set_voxel_inds = set_voxel_inds_tensor_shift_1[set_id:set_id+1].squeeze(0)
        set_voxel_masks = set_voxel_masks_tensor_shift_1[set_id:set_id+1].squeeze(0)
        pos_embed = pos_embed_tensor[blc_id:blc_id+1, set_id:set_id+1].squeeze(0).squeeze(0)
        # inputs = (outputs, set_voxel_inds, set_voxel_masks, pos_embed, True)
        inputs = (outputs, set_voxel_inds, set_voxel_masks, pos_embed)
        outputs = self.dsvtblocks_list[blc_id].encoder_list[set_id](*inputs)
        set_id = 1
        set_voxel_inds = set_voxel_inds_tensor_shift_1[set_id:set_id+1].squeeze(0)
        set_voxel_masks = set_voxel_masks_tensor_shift_1[set_id:set_id+1].squeeze(0)
        pos_embed = pos_embed_tensor[blc_id:blc_id+1, set_id:set_id+1].squeeze(0).squeeze(0)
        inputs = (outputs, set_voxel_inds, set_voxel_masks, pos_embed)
        # inputs = (outputs, set_voxel_inds, set_voxel_masks, pos_embed, True)
        outputs = self.dsvtblocks_list[blc_id].encoder_list[set_id](*inputs)

        outputs = self.layer_norms_list[blc_id](residual + outputs)

        return outputs

In [3]:
# cfg_file = "./onnx_config.yaml"
cfg_file = "../tools/cfgs/waymo_models/dsvt_pillar.yaml"
cfg_from_yaml_file(cfg_file, cfg)
if os.path.exists('./deploy_files_test')==False:
    os.mkdir('./deploy_files_test')
log_file = './deploy_files_test/log_trt.log'
logger = common_utils.create_logger(log_file, rank=0)
test_set, test_loader, sampler = build_dataloader(
    dataset_cfg=cfg.DATA_CONFIG,
    class_names=cfg.CLASS_NAMES,
    batch_size=1,
    dist=False, workers=8, logger=logger, training=False
)

2024-03-09 04:09:43,743   INFO  Loading Custom dataset.
2024-03-09 04:09:43,745   INFO  Total samples for CUSTOM dataset: 0


In [8]:
model = build_network(model_cfg=cfg.MODEL, num_class=len(cfg.CLASS_NAMES), dataset=test_set)


{'ROOT_DIR': PosixPath('/opt/dec'),
 'LOCAL_RANK': 0,
 'CLASS_NAMES': ['Person'],
 'DATA_CONFIG': {'BALANCED_RESAMPLING': False,
  'DATASET': 'CustomDataset',
  'DATA_AUGMENTOR': {'AUG_CONFIG_LIST': [{'DATABASE_WITH_FAKELIDAR': False,
     'DB_INFO_PATH': ['custom_dbinfos_train.pkl'],
     'LIMIT_WHOLE_SCENE': True,
     'NAME': 'gt_sampling',
     'NUM_POINT_FEATURES': 4,
     'PREPARE': {'filter_by_min_points': ['Person:3']},
     'REMOVE_EXTRA_WIDTH': [0.0, 0.0, 0.0],
     'SAMPLE_GROUPS': ['Person:0'],
     'USE_ROAD_PLANE': False},
    {'ALONG_AXIS_LIST': ['x', 'y'], 'NAME': 'random_world_flip'},
    {'NAME': 'random_world_rotation',
     'WORLD_ROT_ANGLE': [-0.78539816, 0.78539816]},
    {'NAME': 'random_world_scaling', 'WORLD_SCALE_RANGE': [0.95, 1.05]}],
   'DISABLE_AUG_LIST': ['placeholder'],
   'MIX': {'ALPHA': 2,
    'COLLISION_DETECTION': True,
    'NAME_LIST': ['mix_up', 'cut_mix'],
    'PROB': 0.3}},
  'DATA_PATH': '/opt/ml/input/data/cache_pointcloud/7b0fb3d3-2c5b-4721-8

In [5]:
ckpt = "/mnt/nas2/users/eslim/result_log/dsvt_pillar_waymo/ckpt/latest_model.pth"
model.load_params_from_file(filename=ckpt, logger=logger, to_cpu=False, pre_trained_path=None)
model.eval()
model.cuda()

2024-03-09 04:09:57,277   INFO  ==> Loading parameters from checkpoint ./pillar/model.pth to GPU
2024-03-09 04:09:57,424   INFO  ==> Checkpoint trained from version: pcdet+0.6.0+255db8f
2024-03-09 04:09:57,457   INFO  ==> Done (loaded 391/391)


CenterPoint(
  (vfe): DynamicVoxelVFE(
    (pfn_layers): ModuleList(
      (0): PFNLayerV2(
        (linear): Linear(in_features=10, out_features=96, bias=False)
        (norm): BatchNorm1d(96, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (relu): ReLU()
      )
      (1): PFNLayerV2(
        (linear): Linear(in_features=192, out_features=192, bias=False)
        (norm): BatchNorm1d(192, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (relu): ReLU()
      )
    )
  )
  (backbone_3d): DSVT(
    (input_layer): DSVTInputLayer(
      (posembed_layers): ModuleList(
        (0): ModuleList(
          (0-3): 4 x ModuleList(
            (0-1): 2 x PositionEmbeddingLearned(
              (position_embedding_head): Sequential(
                (0): Linear(in_features=2, out_features=192, bias=True)
                (1): BatchNorm1d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                (2): ReLU(inplace=True)
       

In [6]:
batch_dict = torch.load("./batch_dict.pth", map_location="cuda")
inputs = batch_dict

In [7]:
with torch.no_grad():
    DSVT_Backbone = model.backbone_3d
    dsvtblocks_list = DSVT_Backbone.stage_0
    layer_norms_list = DSVT_Backbone.residual_norm_stage_0
    inputs = model.vfe(inputs)
    voxel_info = DSVT_Backbone.input_layer(inputs)
    set_voxel_inds_list = [[voxel_info[f'set_voxel_inds_stage{s}_shift{i}'] for i in range(2)] for s in range(1)]
    set_voxel_masks_list = [[voxel_info[f'set_voxel_mask_stage{s}_shift{i}'] for i in range(2)] for s in range(1)]
    pos_embed_list = [[[voxel_info[f'pos_embed_stage{s}_block{b}_shift{i}'] for i in range(2)] for b in range(4)] for s in range(1)]

    pillar_features = inputs['voxel_features']
    alldsvtblockstrt_inputs = (
        pillar_features,
        set_voxel_inds_list[0][0],
        set_voxel_inds_list[0][1],
        set_voxel_masks_list[0][0],
        set_voxel_masks_list[0][1],
        torch.stack([torch.stack(v, dim=0) for v in pos_embed_list[0]], dim=0),
    )

RuntimeError: mat1 and mat2 shapes cannot be multiplied (125027x11 and 10x96)

In [8]:

jit_mode = "trace"
input_names = [
    'src',
    'set_voxel_inds_tensor_shift_0',
    'set_voxel_inds_tensor_shift_1',
    'set_voxel_masks_tensor_shift_0',
    'set_voxel_masks_tensor_shift_1',
    'pos_embed_tensor'
]
output_names = ["output",]
input_shapes = {
    "src": {
        "min_shape": [24629, 192],
        "opt_shape": [24629, 192],
        "max_shape": [24629, 192],
    },
    "set_voxel_inds_tensor_shift_0": {
        "min_shape": [2, 1156, 36],
        "opt_shape": [2, 1156, 36],
        "max_shape": [2, 1156, 36],
    },
    "set_voxel_inds_tensor_shift_1": {
        "min_shape": [2, 834, 36],
        "opt_shape": [2, 834, 36],
        "max_shape": [2, 834, 36],
    },
    "set_voxel_masks_tensor_shift_0": {
        "min_shape": [2, 1156, 36],
        "opt_shape": [2, 1156, 36],
        "max_shape": [2, 1156, 36],
    },
    "set_voxel_masks_tensor_shift_1": {
        "min_shape": [2, 834, 36],
        "opt_shape": [2, 834, 36],
        "max_shape": [2, 834, 36],
    },
    "pos_embed_tensor": {
        "min_shape": [4, 2, 24629, 192],
        "opt_shape": [4, 2, 24629, 192],
        "max_shape": [4, 2, 24629, 192],
    },
}

dynamic_axes = {
    "src": {
        0: "voxel_number",
    },
    "set_voxel_inds_tensor_shift_0": {
        1: "set_number_shift_0",
    },
    "set_voxel_inds_tensor_shift_1": {
        1: "set_number_shift_1",
    },
    "set_voxel_masks_tensor_shift_0": {
        1: "set_number_shift_0",
    },
    "set_voxel_masks_tensor_shift_1": {
        1: "set_number_shift_1",
    },
    "pos_embed_tensor": {
        2: "voxel_number",
    },
    "output": {
        0: "voxel_number",
    }
}

In [9]:
base_name = "./deploy_files_test/dsvt"
ts_path = f"{base_name}.ts"
onnx_path = f"{base_name}.onnx"

allptransblocktrt = AllDSVTBlocksTRT(dsvtblocks_list, layer_norms_list).eval().cuda()

torch.onnx.export(
    allptransblocktrt,
    alldsvtblockstrt_inputs,
    onnx_path, input_names=input_names,
    output_names=output_names, dynamic_axes=dynamic_axes,
    opset_version=14,
)
# https://github.com/Haiyang-W/DSVT/issues/60

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [12]:
# test onnx
ort_session = ort.InferenceSession(onnx_path)
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# compute ONNX Runtime output prediction
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(pillar_features),
              ort_session.get_inputs()[1].name: to_numpy(set_voxel_inds_list[0][0]),
              ort_session.get_inputs()[2].name: to_numpy(set_voxel_inds_list[0][1]),
              ort_session.get_inputs()[3].name: to_numpy(set_voxel_masks_list[0][0]),
              ort_session.get_inputs()[4].name: to_numpy(set_voxel_masks_list[0][1]),
              ort_session.get_inputs()[5].name: to_numpy(torch.stack([torch.stack(v, dim=0) for v in pos_embed_list[0]], dim=0)),}
ort_outs = ort_session.run(None, ort_inputs)
ort_outs


[array([[-0.00120146,  0.0012869 ,  0.13761668, ..., -0.04611877,
          0.005659  , -0.02467331],
        [-0.00172799, -0.00854082, -0.05844514, ..., -0.08249708,
          0.00571191,  0.03585856],
        [-0.00201712,  0.02557927,  0.12964977, ..., -0.07701403,
          0.00571428,  0.2548802 ],
        ...,
        [-0.00198825,  0.03332692,  0.07325681, ..., -0.07479647,
          0.00561676, -0.14559093],
        [-0.00226119, -0.01397068,  0.13979352, ...,  0.11101341,
          0.00556513, -0.24747433],
        [-0.00113338,  0.02863011, -0.04145716, ...,  0.05549115,
          0.00571733, -0.25773776]], dtype=float32)]